# Introduction to Creating Tables

To try out how to create tables and insert data into the tables we are using [duckdb](https://duckdb.org/) in this notebook. It is creating a local database which you can run SQL-queries in Jupyter Notebooks against. DBeaver also has a connector so you can also run SQL-queries in there. Duckdb works with [sqlalchemy](https://www.sqlalchemy.org/) a widely used python library to connect to databases and ipython-sql. Ipython-sql allows us to convert a Jupyter code cell into a SQL cell with the jupyter magic command ```%%sql```.

In this notebook you will learn how to create tables and insert data into the created tables.

In [ ]:

import duckdb
import pandas as pd
import sqlalchemy
import json
# No need to import duckdb_engine
# SQLAlchemy will auto-detect the driver needed bASed on your connection string!

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql

Set configrations on ipython-sql to directly output data to Pandas and to simplify the output that is printed to the notebook.

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Connect ipython-sql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.

In [ ]:
%sql duckdb:///sampleDB.duckdb

We have two csv files in our data directory:
- raw_orders.csv
- raw_payments.csv

and a json file:
- raw_customers.json


So we will see how to insert csv files and json files into our database but first we will create a tables for the data and for that let's have a look at the data and headers.

In [ ]:
df_customers = pd.read_json('data/raw_customers.json', orient='records', lines=True)
df_customers.head()

In [ ]:
df_orders = pd.read_csv('data/raw_orders.csv')
df_orders.head()

In [ ]:
df_payment = pd.read_csv('data/raw_payments.csv')
df_payment.head()

Let us start with creating the tables, for that it is standard to define the column types. The column types can differ from sql flavour to sql flavour.

We will start with the ```raw_cusomer``` table here we have:
- id
- first_name
- last_name

We will also define the ```id``` as ```INTEGER``` and ```PRIMARY KEY``` and additionally we say that it can't have missing values with ```NOT NULL```. The other two columns are strings which is call ```VARCHAR``` in sql.

In [ ]:
%%sql

CREATE TABLE raw_customers(
    id INTEGER PRIMARY KEY NOT NULL,
    first_name VARCHAR,
    last_name VARCHAR
    );


Next we will create the ```raw_orders``` table:
- id
- user_id
- order_date
- status

We will define the ```id``` as ```INTEGER``` and ```PRIMARY KEY``` and additionally we say that it can't have missing values with ```NOT NULL```. The ```user_id``` is a ```FOREIGN KEY``` and references the raw_customer id. Order_date is a ```DATE``` and status is a ```VARCHAR```.

In [ ]:
%%sql

CREATE TABLE raw_orders(
    id INTEGER PRIMARY KEY NOT NULL,
    user_id INTEGER,
    FOREIGN KEY(user_id) REFERENCES raw_customers(id),
    order_date DATE,
    status VARCHAR
    );

Last we will create the ```raw_payments``` table:
- id
- order_id
- payment_method
- amount

Again we will define the ```id``` as ```INTEGER``` and ```PRIMARY KEY``` and additionally we say that it can't have missing values with ```NOT NULL```. The ```order_id``` is a ```FOREIGN KEY``` and references the raw_orders id. payment_method is a ```VARCHAR``` and amount is a ```INTEGER```.

In [ ]:
%%sql

CREATE TABLE raw_payments(
    id INTEGER PRIMARY KEY NOT NULL,
    order_id INTEGER,
    FOREIGN KEY(order_id) REFERENCES raw_orders(id),
    payment_method VARCHAR,
    amount INTEGER
    );

So now we created the tables but at the moment they are empty. Let's check that.

In [ ]:
%%sql

select * from raw_customers

In [ ]:
%%sql

select * from raw_orders

In [ ]:
%%sql

select * from raw_payments

As expected they are empty, so let's change that and use ```INSERT``` for that. Most sql flavours have build in functions to deal with external data sources like csv or json files. In duckdb we can simply use ```read_csv_auto``` to read our csv files and we can than run sql queries against that. For json it is a little bit more complicated we have to create first a table where we store the json itself.

For json we we have to install and load the ```json``` extension from duckdb.

In [ ]:
%%sql

INSTALL 'json';
LOAD 'json';

Now we have to create the table we store the json in. This table has only one column ```j``` with the data type ```JSON```.

In [ ]:
%%sql

CREATE TABLE raw_customers_json(
    j JSON
    );


Now we can use the ```INSERT INTO``` command followed by a select statement where the json is read with ```read_json_objects()```.

In [ ]:
%%sql

INSERT INTO raw_customers_json SELECT * FROM read_json_objects('data/raw_customers.json', format='auto')

Let's have a look if that works:

In [ ]:
%%sql

SELECT * FROM raw_customers_json

Now that we have the json in a database we can parse it and put it into our ```raw_customer``` table, we can parse the json with the ```json_extract``` function with the json and the json_path as input e.g. ```json_extract(j, '$.id')```.
```json
{"id":1,"first_name":"Michael","last_name":"P."}
```
The example above will get the ```id``` 1.

We will also use the ```CAST``` function which converts a value (of any type) into a specified datatype. This will be included in a ```INSERT INTO``` and ```SELECT``` statement. And because there are quotationmarks around the names we ```REPLACE``` them.

In [ ]:
%%sql
INSERT INTO raw_customers
SELECT 
    CAST(json_extract(j, '$.id') AS INTEGER) AS id,
    REPLACE(CAST(json_extract(j, '$.first_name')AS VARCHAR),'"','') AS first_name, 
    REPLACE(CAST(json_extract(j, '$.last_name')AS VARCHAR),'"','') AS last_name 
FROM raw_customers_json;

Inserting the data from the csv files is a little bit easier with the ```read_csv_auto()``` function. 

First for the orders:

In [ ]:
%%sql

INSERT INTO raw_orders SELECT id,user_id,order_date,status FROM read_csv_auto('data/raw_orders.csv');

And for the payments:

In [ ]:
%%sql

INSERT INTO raw_payments SELECT id,order_id,payment_method,amount FROM read_csv_auto('data/raw_payments.csv');

Now let's have a look into the data with SQL:

In [ ]:
%%sql

SELECT * FROM raw_customers

In [ ]:
%%sql

SELECT * FROM raw_orders

In [ ]:
%%sql

SELECT * FROM raw_payments